In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.animation as animation

# Pytorch packages
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
import torch.nn.functional as F

## Setting up the GPU if available

In [4]:
# Device selection

GPU = True
device_idx = 0
if GPU:
    device = torch.device('cuda'+str(device_idx) if torch.cuda.is_available() else 'cpu')
else:
    print('please first activate the GPU by going Runtime>Change Runtime type')
    device = torch.device('cpu')
print('The device that will be used is:',device)

The device that will be used is: cpu


In [6]:
transform = transforms.Compose(
    [
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ]
)

'''
torchvision.datasets.ImageFolder is a generic data loader where the images are arranged in this way:

root/dog/xxx.png
root/dog/xxy.png
root/dog/xxz.png

root/cat/123.png
root/cat/nsdf3.png
root/cat/asd932_.png'''

path = './NaturalImageNet'
dataset = datasets.ImageFolder(root=path, transform=transform)

In [7]:
# Create train test split
n = len(dataset)
n_test = 1000 # use 1000 images as test set

train_set, test_set = torch.utils.data.random_split(dataset, [n-n_test, n_test])

# The number of image to processs in one go. If you run out of GPU memory reduce this number
batch_size = 128

loader_train = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
loader_test = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=4)